In [2]:
# 39 Mizuno
# 36 Joma
# 28 Skechers
# 21 Asics
# 20 Saucony

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_data():
    url = "https://bigfootsport.pt/lista_artigos_sf.php?marca=60&nmarca=MIZUNO"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    name_with_color = []
    product_name = soup.find_all('div', class_='linha1fix')
    if product_name:
        for product_element in product_name:
            name_with_color.append(product_element.text)
    else:
        print("Product names not found")

    known_colors = ["Preto", 'Preta', "Azul", "Vermelho", 'Vermelha', "Verde", "Amarelo", 'Amarela', "Branco", 'Branca', "Laranja", "Rosa", "Cinza", 'Bo', 'Roxo', 'Roxa', 'Br', 'Brancas', 'Wos','Lightning Star Jr']

    def extract_colors(product_names, known_colors):
        extracted_colors = []
        modified_names = []  
        for product in product_names:
            for color in known_colors:
                if color in product:
                    product = product.replace(color, "").strip()
                    extracted_colors.append(color)
                    modified_names.append(product)
                    break
        return extracted_colors, modified_names
    
    known_models = ['Mizuno Morelia', 'Core', 'Joelheira', 'Meias', 'Wave', 'Thunder', 'Padel', 'Lightning', 'Voltage', 'Manguito', 'Perneira', 'Meia']

    def extract_model(product_names, known_models):
        extracted_model = []
        for product in product_names:
            for model in known_models:
                if model in product:
                    extracted_model.append(model)
                    break
        extracted_model.insert(12, 'Wave')
        extracted_model.insert(-1, 'Calçao')
        extracted_model.insert(-1, 'Calçao')
        extracted_model.insert(-1, 'Calçao')
        return extracted_model

    extracted_colors, modified_product_names = extract_colors(name_with_color, known_colors)

    price_elements = soup.find_all('span', style=lambda value: value and 'text-decoration: line-through' in value)

    struck_price = []
    if price_elements:
        for price_element in price_elements:
            struck_price.append(price_element.text[:-1])
    else:
        print("Price elements not found")

    discount_values = soup.find_all('font', {'color': '#000'})
    discount_list = []

    for element in discount_values:
        discount_list.append(element.text[:-1])

    min_length = min(len(modified_product_names), len(extracted_colors), len(struck_price), len(discount_list))
    extracted_model =  extract_model(name_with_color, known_models)
    
    modified_product_names = modified_product_names[:min_length]
    extracted_colors = extracted_colors[:min_length]
    struck_price = struck_price[:min_length]
    discount_list = discount_list[:min_length]
    brand = 'Mizuno'
    gender = ["men", "men", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "men", "men", "men", "kid", "kid", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "women", "men", "men", "men", "men", "men", "men", "men", "men", "men", "men", "kid", "men", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One", "Another_One"]
    
    df = pd.DataFrame({
        'brand': brand,
        'primary_color': extracted_colors,
        'estimatedMarketValue': discount_list,
        'gender' : gender,
        'name': modified_product_names,
        'retailPrice': struck_price,
        'silhouette': extracted_model,
    })

    return df

df = scrape_product_data()

df.to_csv('sneakers_mizuno.csv', index=False)
